In [1]:
import numpy as np
import itertools
import pickle
import os.path

In [2]:
# Function which checks whether the input state is a winning state.
# playerIdx = 1 means the player is 'X', playerIdx = 2 means the player is 'O'
def isWinningState(inState, playerIdx=1):
    
    # Check for horizontal win states
    idx1 = 0
    idx2 = 1
    idx3 = 2
    for i in xrange(3):
        if (inState[idx1] == inState[idx2] == inState[idx3]) and inState[idx1] == playerIdx:
            return True
        
        idx1 = idx1 + 3
        idx2 = idx2 + 3
        idx3 = idx3 + 3
    
    # Check for vertical win states
    idx1 = 0
    idx2 = 3
    idx3 = 6
    
    for i in xrange(3):
        if (inState[idx1] == inState[idx2] == inState[idx3]) and inState[idx1] == playerIdx:
            return True
        
        idx1 = idx1 + 1
        idx2 = idx2 + 1
        idx3 = idx3 + 1
    
    # Check for diagonal win state 1
    if (inState[0] == inState[4] == inState[8]) and inState[0] == playerIdx:
        return True
    
    # Check for diagonal win state 2
    if (inState[2] == inState[4] == inState[6]) and inState[2] == playerIdx:
        return True
    
    # If we got to here, no win states were found
    return False
            

In [3]:
# Checks if this is a drawn state.
# Note that this should be called AFTER checking if 
# this state is a winning state because this does no
# explicit win check
def isDrawn(state):
    for i in xrange(9):
        if state[i] == 0:
            return False
        
    # If we get to here then it is a drawn game
    return True

In [4]:
# Determines possible next states for the computer
def nextStates(state, playerIdx=1):
    nextStates = []
    stateList = list(state)
    for i in xrange(9):
        if stateList[i] == 0:
            newState = list(stateList)
            newState[i] = playerIdx
            nextStates.append(tuple(newState))
            
    return nextStates

In [5]:
# Takes a state as a tuple, e.g. (1, 0, 0, 1, 1, 0, 0, 0, 0)
# and prints out the appearance of the board, e.g:
#
#           X  |      |    
#           X  |   X  |    
#              |      |
def printBoardState(state):
    idx = 0
    lookup = {0: ' ', 1: 'X', 2: 'O'}
    for i in xrange(3):
        c0 = lookup[state[idx]]
        c1 = lookup[state[idx+1]]
        c2 = lookup[state[idx+2]]
        if c0 == ' ':
            c0 = str(idx)
        if c1 == ' ':
            c1 = str(idx + 1)
        if c2 == ' ':
            c2 = str(idx + 2)
        print c0 + '\t|\t' + c1 + '\t|\t' + c2
        idx = idx + 3

In [6]:
# Our computer will play player 1, i.e. it will play as 'X'
playerIdx = 1
# Create lookup table for all possible board states and set initial values to zero
V = {}
for combination in itertools.product(xrange(3), repeat=9):
    if (isWinningState(combination, playerIdx)):
        V[combination] = 1.0
    else:
        V[combination] = 0.0

In [28]:
nTrainingEpisodes = 100000
stepSize = 0.000000001
exploreProbability = 0.1
trainFromScratch = True
trainAtAll = True

# Load in a previously trained value table
if not trainFromScratch and os.path.exists('valuetable.txt'):
    with open('valuetable.txt', 'rb') as handle:
        V = pickle.loads(handle.read())
        
tally1 = 0
tally2 = 0

if trainAtAll:
    for i in xrange(nTrainingEpisodes):
        # Initialise the game
        curState = (0,0,0,0,0,0,0,0,0)
        while True:
            futureStates = nextStates(curState)
            nFutureStates = len(futureStates)
            # Explore suboptimal states with some probability
            if np.random.uniform() < exploreProbability:
                
                # Select a random state to explore
                randomStateIdx = int(nFutureStates * np.random.uniform())
                curState = futureStates[randomStateIdx]
                tally1 = tally1 + 1
            else:
                # Iterate through future states and find best one
                # in terms of value function, V
                maxValue = V[futureStates[0]]
                maxIdx = 0
                for j in xrange(nFutureStates):
                    if V[futureStates[j]] > maxValue:
                        maxValue = V[futureStates[j]]
                        maxIdx = j

                # Perform Temporal Difference update: V(s) = V(s) + stepSize*[V(s') - V(s)]
                newState = futureStates[maxIdx]
                V[curState] = V[curState] + stepSize * (V[newState] - V[curState]) 
                curState = newState
                tally2 = tally2 + 1

            # Check if the game is over
            if isWinningState(curState, 1):
                break

            # Check if game is drawn
            if isDrawn(curState):
                break

            # NOW PLAY AS THE OTHER PLAYER (i.e. player 2)
            futureStates = nextStates(curState, 2)
            nFutureStates = len(futureStates)
            # Explore suboptimal states with some probability
            if np.random.uniform() < exploreProbability:
                # Select a random state to explore
                randomStateIdx = int(nFutureStates * np.random.uniform())
                curState = futureStates[randomStateIdx]
            else:
                # Iterate through future states and find best one
                # in terms of value function, V
                maxValue = V[futureStates[0]]
                maxIdx = 0
                for j in xrange(nFutureStates):
                    if V[futureStates[j]] > maxValue:
                        maxValue = V[futureStates[j]]
                        maxIdx = j

                # Perform Temporal Difference update: V(s) = V(s) + stepSize*[V(s') - V(s)]
                newState = futureStates[maxIdx]
                V[curState] = V[curState] + stepSize * (V[newState] - V[curState]) 
                curState = newState

            # Check if the game is over
            if isWinningState(curState, 2):
                break

            # Check if game is drawn
            if isDrawn(curState):
                break
    
    print 'Expored ' + str(tally1) + ' times'
    print 'Learned ' + str(tally2) + ' times'

    # Save the value table to file
    with open('valuetable.txt', 'wb') as handle:
        pickle.dump(V, handle)

Expored 31349 times
Learned 283042 times


In [24]:
V

{(1, 2, 1, 2, 2, 2, 0, 1, 0): 0.0,
 (1, 2, 1, 1, 2, 0, 1, 2, 0): 1.0,
 (2, 1, 2, 2, 1, 1, 1, 1, 1): 1.0,
 (1, 2, 0, 2, 2, 1, 1, 0, 1): 0.0008331775543931764,
 (2, 1, 0, 0, 2, 1, 2, 0, 2): 0.0,
 (0, 0, 1, 1, 1, 2, 2, 0, 0): 8.779454947475026e-12,
 (0, 2, 2, 0, 1, 2, 2, 0, 1): 0.0,
 (2, 0, 2, 2, 1, 0, 2, 2, 2): 0.0,
 (2, 0, 1, 0, 1, 2, 0, 2, 0): 0.0,
 (2, 2, 2, 2, 0, 0, 0, 0, 1): 0.0,
 (2, 0, 2, 1, 0, 1, 1, 1, 1): 1.0,
 (0, 0, 2, 0, 1, 0, 1, 1, 0): 0.0,
 (0, 2, 0, 1, 1, 2, 1, 1, 1): 1.0,
 (2, 1, 1, 2, 2, 1, 2, 2, 0): 0.0,
 (1, 0, 2, 0, 0, 1, 0, 2, 2): 0.0,
 (2, 0, 2, 2, 2, 0, 1, 0, 0): 0.0,
 (2, 1, 2, 0, 1, 2, 2, 2, 0): 0.0,
 (0, 2, 0, 1, 0, 0, 1, 0, 0): 0.005457067755353132,
 (2, 0, 0, 2, 2, 1, 0, 0, 0): 0.0,
 (2, 1, 0, 2, 2, 0, 1, 1, 0): 0.04703391984475707,
 (0, 2, 0, 2, 0, 0, 1, 1, 1): 1.0,
 (2, 0, 0, 2, 1, 0, 1, 0, 2): 0.0,
 (0, 1, 0, 1, 1, 2, 0, 1, 1): 1.0,
 (1, 1, 1, 1, 2, 0, 0, 0, 0): 1.0,
 (0, 0, 2, 1, 0, 1, 1, 2, 1): 0.0,
 (1, 0, 0, 0, 1, 0, 1, 1, 2): 0.0,
 (0, 2, 1, 1, 2, 2, 0

In [25]:
# ********* Play against the computer **********
# Initialise the game
curState = (0,0,0,0,0,0,0,0,0)
while True:
    # Iterate through future states and find best one
    # in terms of value function, V
    futureStates = nextStates(curState)
    nFutureStates = len(futureStates)
    maxValue = V[futureStates[0]]
    maxIdx = 0
    for j in xrange(nFutureStates):
        if V[futureStates[j]] > maxValue:
            maxValue = V[futureStates[j]]
            maxIdx = j

    curState = futureStates[maxIdx]
    
    printBoardState(curState)

    # Check if the game is over
    if isWinningState(curState, 1):
        print 'Game Over! Bot Wins!'
        break

    # Check if game is drawn
    if isDrawn(curState):
        print 'Game Over! Drawn.'
        break
    
    # Ask the user what their move is
    userMoveIdx = raw_input('Please enter your move: ')
    curStateList = list(curState)
    curStateList[int(userMoveIdx)] = 2
    curState = tuple(curStateList)
    
        # Check if the game is over
    if isWinningState(curState, 2):
        print 'Game Over! You Win!'
        break

    # Check if game is drawn
    if isDrawn(curState):
        print 'Game Over! Drawn.'
        break
        
    

X	|	1	|	2
3	|	4	|	5
6	|	7	|	8
Please enter your move: 4
X	|	1	|	2
X	|	O	|	5
6	|	7	|	8
Please enter your move: 6
X	|	X	|	2
X	|	O	|	5
O	|	7	|	8
Please enter your move: 2
Game Over! You Win!


In [9]:
# How to load back the saved value table
with open('valueable.txt', 'rb') as handle:
  b = pickle.loads(handle.read())

IOError: [Errno 2] No such file or directory: 'valueable.txt'

In [12]:
os.path.exists('valuetble.txt')

False